In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nitishpandey04","key":"25e697bcb9f519cb2af5d5a7c19f2874"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

 98% 794M/812M [00:03<00:00, 240MB/s]
100% 812M/812M [00:03<00:00, 245MB/s]


In [ ]:
!unzip -qq dogs-vs-cats.zip

In [ ]:
!unzip -qq train.zip

In [ ]:
import os, shutil, pathlib

In [ ]:
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

In [ ]:
def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f"{category}.{i}.jpg"
                  for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2500)

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=32
)

validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=32
)

test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=32
)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


### We will train the model with EarlyStopping callback along with ModelCheckpoint and we do data augmentation. 

In [ ]:
data_augmentation = keras.Sequential([
                                      layers.RandomFlip("horizontal"),
                                      layers.RandomRotation(0.1),
                                      layers.RandomZoom(0.2)
])

In [ ]:
callbacks = [
            #  keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
             keras.callbacks.ModelCheckpoint(
                 filepath="regularized_covnet_from_scratch.keras",
                 save_best_only=True,
                 monitor="val_loss"
             )
]

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
history = model.fit(train_dataset,
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=validation_dataset)

Epoch 1/100
63/63 [==============================] - 19s 125ms/step - loss: 0.7107 - accuracy: 0.5215 - val_loss: 0.6892 - val_accuracy: 0.5230
Epoch 2/100
63/63 [==============================] - 7s 112ms/step - loss: 0.6878 - accuracy: 0.5695 - val_loss: 0.6500 - val_accuracy: 0.6160
Epoch 3/100
63/63 [==============================] - 7s 113ms/step - loss: 0.6753 - accuracy: 0.6065 - val_loss: 0.6689 - val_accuracy: 0.5260
Epoch 4/100
63/63 [==============================] - 7s 111ms/step - loss: 0.6448 - accuracy: 0.6610 - val_loss: 0.6212 - val_accuracy: 0.6550
Epoch 5/100
63/63 [==============================] - 7s 111ms/step - loss: 0.5958 - accuracy: 0.6785 - val_loss: 0.5840 - val_accuracy: 0.6760
Epoch 6/100
63/63 [==============================] - 7s 110ms/step - loss: 0.5601 - accuracy: 0.7020 - val_loss: 0.5746 - val_accuracy: 0.6840
Epoch 7/100
63/63 [==============================] - 7s 111ms/step - loss: 0.5492 - accuracy: 0.7200 - val_loss: 0.5702 - val_accuracy: 0.697

In [ ]:
saved_model = keras.models.load_model("regularized_covnet_from_scratch.keras")

In [ ]:
saved_model.evaluate(test_dataset)

63/63 [==============================] - 4s 48ms/step - loss: 0.5855 - accuracy: 0.7300


[0.5855461359024048, 0.7300000190734863]